<a href="https://colab.research.google.com/github/ErikaNat/Inteligencia-Artificial/blob/main/EJERCICIODEPRACTICA7CANCERDEMAMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#==========================================
# 0) Imports y Configuracion  hacer prediccion, implementarla en esta bnueva red neuronal, con mismas librerias, para red secienciasl y base de preprocesamiento, la digerencia es que  no hay prediccion, se entrena,

#==========================================
import os
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, accuracy_score

from tensorflow.keras import layers, models, callbacks
from sklearn.pipeline import Pipeline


#Reproductibilidad
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

print("TensorFlow:", tf.__version__)
print("Keras:", tf.keras.__version__)

TensorFlow: 2.19.0
Keras: 3.10.0


In [ ]:
#==========================================
#1) Cargar dataset Breast Cancer
#==========================================

FILE_PATH = "/content/Breast_cancer_dataset.csv"
df = pd.read_csv(FILE_PATH)
display(df.head())

#Mirada Rapida
print("Shape bruto:" , df.shape)
print("Cols", list(df.columns))

#============================
#2) Seleccion de variables and Target Mapping
#============================
#OObjetivo;diagnosis (0/1)
# Map 'M' to 1 and 'B' to 0
df['diagnosis'] = df['diagnosis'].map({'M': 1, 'B': 0})
y = df["diagnosis"].astype(int).values

#Tomamos columnnas  usuales y evitamos leakage (id no ayuda de base)
X = df[[
    "radius_mean", "texture_mean", "perimeter_mean", "area_mean", "concavity_mean", "radius_se", "texture_se",
    "perimeter_se", "area_se", "concavity_se", "radius_worst", "texture_worst", "perimeter_worst", "area_worst",
    "concavity_worst"
]].copy()



#=======================================
#3) Definicion de columnas por tipo
#=======================================
# Based on the selected columns, all appear to be numerical.
# We will confirm this during preprocessing.
num_cols = [
    "radius_mean", "texture_mean", "perimeter_mean", "area_mean", "concavity_mean", "radius_se", "texture_se",
    "perimeter_se", "area_se", "concavity_se", "radius_worst", "texture_worst", "perimeter_worst", "area_worst",
    "concavity_worst"
]
cat_cols = [] # No apparent categorical columns in the selected features.

#

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


Shape bruto: (569, 33)
Cols ['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean', 'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se', 'fractal_dimension_se', 'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst', 'concave points_worst', 'symmetry_worst', 'fractal_dimension_worst', 'Unnamed: 32']


In [ ]:
#===============================
#4) Preprocesamiento con columnTransformer
#===============================
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

from sklearn.pipeline import Pipeline
preprocess = ColumnTransformer(transformers=[
    ("num", numeric_transformer, num_cols),
    ("cat", categorical_transformer, cat_cols),
],
                               remainder="drop"
                               )



In [ ]:
#============================
#5) Split tran/Test estratificado
#=============================
X_train_df, X_test_df, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
    )
#Ajustar transformadores en train y transformer ambos
X_train = preprocess.fit_transform(X_train_df)
X_test = preprocess.transform(X_test_df)

print("Imput dims:", X_train.shape[1])

Imput dims: 15


In [ ]:
#===================================
#6) Definir y compilar  el modelo
#===================================
def build_model(imput_dim: int) -> tf.keras.Model:
    model = models.Sequential([
        layers.Input(shape=(imput_dim,)),
        layers.Dense(32, activation="relu"),
        layers.Dropout(0.15),#desactiva ese porcentaje de neuronas 15% , para no acomplejar
        layers.Dense(32, activation="relu"),#32 nueronas
        layers.Dropout(0.15),
        layers.Dense(1, activation="sigmoid")#una neurona de salida
    ])
    model.compile(
        optimizer="adam",
        loss="binary_crossentropy",
        metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
    )
    return model

model = build_model(X_train.shape[1])
model.summary()


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                │ (None, 32)             │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,601 (6.25 KB)

 Trainable params: 1,601 (6.25 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# ================================
# 7) Callbacks (buenoas prácticas)
# ================================
cbs = [
    callbacks.EarlyStopping(monitor="val_auc", mode="max", patience=12, restore_best_weights=True),
    callbacks.ModelCheckpoint("titanic_best.keras", monitor="val_auc", mode="max", save_best_only=True),
    callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=6)
]

In [ ]:
# ====================
# 8) Entrenamiento
# ====================
hist = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=32,
    callbacks=cbs,
    verbose=1
)

Epoch 1/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - accuracy: 0.6882 - auc: 0.7668 - loss: 0.5812 - val_accuracy: 0.8022 - val_auc: 0.9281 - val_loss: 0.4336 - learning_rate: 0.0010
Epoch 2/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8747 - auc: 0.9486 - loss: 0.3991 - val_accuracy: 0.8901 - val_auc: 0.9590 - val_loss: 0.3238 - learning_rate: 0.0010
Epoch 3/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9267 - auc: 0.9721 - loss: 0.3066 - val_accuracy: 0.9011 - val_auc: 0.9727 - val_loss: 0.2627 - learning_rate: 0.0010
Epoch 4/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9329 - auc: 0.9831 - loss: 0.2495 - val_accuracy: 0.9121 - val_auc: 0.9749 - val_loss: 0.2263 - learning_rate: 0.0010
Epoch 5/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9339 - auc: 0.9780 - loss: 0.2243 - val_accuracy: 0.9341 - val_auc: 0.9770 - val_loss: 0.2022 - learning_rate: 0.0010
Epoch 6/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9527 - auc: 0.

In [ ]:
# ==========================
# 9) Evaluación  en test
# ==========================
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

# 1. UsaR EL MODELO ENTRNADO
y_proba = model.predict(X_test).ravel()

# 2. Convertir
y_pred = (y_proba > 0.5).astype(int)

# 3. CCalcular matriz
print("\nMatriz de confusión:\n", confusion_matrix(y_test, y_pred))

# 4. Imprimir reporte
print("\nReporte de clasificación:\n", classification_report(y_test, y_pred, digits=4))

# 5. Calcuar AUC
print("\nAUC:", roc_auc_score(y_test, y_proba))

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step

Matriz de confusión:
 [[71  1]
 [ 4 38]]

Reporte de clasificación:
               precision    recall  f1-score   support

           0     0.9467    0.9861    0.9660        72
           1     0.9744    0.9048    0.9383        42

    accuracy                         0.9561       114
   macro avg     0.9605    0.9454    0.9521       114
weighted avg     0.9569    0.9561    0.9558       114


AUC: 0.9950396825396826


In [ ]:
# =============================================================
# 10) Interferencia robusta: función predict_one(dict de entrada)
# =============================================================
def predict_one(sample: dict) -> float:
    """
    Recibe un diccionario 'crudo' con las llaves esperadas:
    radius_mean, texture_mean, perimeter_mean, area_mean, concavity_mean,
    radius_se, texture_se, perimeter_se, area_se, concavity_se,
    radius_worst, texture_worst, perimeter_worst, area_worst, concavity_worst.
    Devuelve la probabilidad de malignidad (diagnosis = 1).
    """
    # Convertir a DataFrame con columnas en orden esperado
    # Ensure the keys in the sample dictionary match the columns in X
    expected_cols = [
        "radius_mean", "texture_mean", "perimeter_mean", "area_mean", "concavity_mean", "radius_se", "texture_se",
        "perimeter_se", "area_se", "concavity_se", "radius_worst", "texture_worst", "perimeter_worst", "area_worst",
        "concavity_worst"
    ]
    # Reordenar diccionario
    ordered_sample = {col: sample.get(col) for col in expected_cols}
    s = pd.DataFrame([ordered_sample])

    # aplicar mismo proessamiento
    # procesar solo columnas numeriacss
    s_proc = preprocess.transform(s) # transformar en s
    s_proc = s_proc.astype("float32")

    # Predecir
    proba = model.predict(s_proc).item()
    return proba

# Ejemplo de inferencia
sample = {
    "radius_mean": 14.23,
    "texture_mean": 19.54,
    "perimeter_mean": 91.15,
    "area_mean": 633.1,
    "concavity_mean": 0.1033,
    "radius_se": 0.4147,
    "texture_se": 1.032,
    "perimeter_se": 2.932,
    "area_se": 42.76,
    "concavity_se": 0.02122,
    "radius_worst": 16.35,
    "texture_worst": 25.53,
    "perimeter_worst": 105.2,
    "area_worst": 802.7,
    "concavity_worst": 0.2575
}

proba = predict_one(sample)
print(f"\nProbabilidad de malignidad (sample): {proba:.4f}")
print("Diagnóstico: Maligno" if proba >= 0.5 else "Diagnóstico: Benigno")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step

Probabilidad de malignidad (sample): 0.4214
Diagnóstico: Benigno
